In [1]:
import data
import tensorflow as tf
from tensorflow.contrib import rnn

In [2]:
lr = 0.001                 # learning rate
batch_size = 70            
n_inputs = 8               # 8 by 8 pixels
n_steps = 8                # time steps
n_hidden_units = 64       # neurons in hidden layer
n_classes = 10             # classes (0-9 digits)

train_data, train_labels, test_data, test_labels = data.load_all_data('data')
train_labels = tf.keras.utils.to_categorical(train_labels, n_classes)
test_labels = tf.keras.utils.to_categorical(test_labels, n_classes)
# x y placeholder
x = tf.placeholder("float", [None, n_steps, n_inputs])
y = tf.placeholder("float", [None, n_classes])

out_weights=tf.Variable(tf.random_normal([n_hidden_units,n_classes]))
out_bias=tf.Variable(tf.random_normal([n_classes]))

In [3]:
test_data.shape

(4000, 64)

In [3]:
#processing the input tensor from [batch_size,n_steps,n_input] to "time_steps" number of [batch_size,n_input] tensors
input=tf.unstack(x ,n_steps, 1)

#defining the network
lstm_layer=rnn.BasicLSTMCell(n_hidden_units,forget_bias=1)
outputs,_=rnn.static_rnn(lstm_layer,input,dtype="float32")

#converting last output of dimension [batch_size,num_units] to [batch_size,n_classes] by out_weight multiplication
prediction=tf.matmul(outputs[-1],out_weights)+out_bias

#loss_function
loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=prediction,labels=y))
#optimization
opt=tf.train.AdamOptimizer(learning_rate=lr).minimize(loss)

#model evaluation
correct_prediction=tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

In [5]:
#initialize variables
init=tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    iter=1
    batch_num=1
    while iter<5001:
        indst = batch_size*(batch_num-1)
        indm = batch_size*batch_num
        batch_x,batch_y = train_data[indst:indm], train_labels[indst:indm]
        
        batch_x=batch_x.reshape((batch_size,n_steps,n_inputs))

        sess.run(opt, feed_dict={x: batch_x, y: batch_y})
        
        batch_num+=1
        if batch_num%7==0:
            batch_num = 1

        if iter%500==0:
            acc=sess.run(accuracy,feed_dict={x:batch_x,y:batch_y})
            los=sess.run(loss,feed_dict={x:batch_x,y:batch_y})
            print("For iter {}, Train Accuracy: {}, Loss: {}".format(iter, acc, los))
            #calculating test accuracy
            test_data = test_data.reshape((-1, n_steps, n_inputs))
            print("Test Accuracy:", sess.run(accuracy, feed_dict={x: test_data, y: test_labels}))

        iter+=1

For iter 500, Train Accuracy: 1.0, Loss: 0.051124703139066696
Test Accuracy: 0.8615
For iter 1000, Train Accuracy: 1.0, Loss: 0.00891096144914627
Test Accuracy: 0.873
For iter 1500, Train Accuracy: 1.0, Loss: 0.001817753305658698
Test Accuracy: 0.8765
For iter 2000, Train Accuracy: 1.0, Loss: 0.001462270156480372
Test Accuracy: 0.8795
For iter 2500, Train Accuracy: 1.0, Loss: 0.0008163460297510028
Test Accuracy: 0.884
For iter 3000, Train Accuracy: 1.0, Loss: 0.00030421471456065774
Test Accuracy: 0.886
For iter 3500, Train Accuracy: 1.0, Loss: 0.00037405002512969077
Test Accuracy: 0.88675
For iter 4000, Train Accuracy: 1.0, Loss: 0.00021856537205167115
Test Accuracy: 0.8875
For iter 4500, Train Accuracy: 1.0, Loss: 9.48695233091712e-05
Test Accuracy: 0.88725
For iter 5000, Train Accuracy: 1.0, Loss: 0.0001334593544015661
Test Accuracy: 0.89025
